# Caipora Project

__Objetivo__




 
__Data Source__

https://queimadas.dgi.inpe.br/queimadas/portal

https://ipsamazonia.org.br/

https://openaq.org/


__Data characteristics__

- Time Series;
- Geographic coordinates – Latitude/Longitude;
- Satellite Name

In [1]:
import os
import math
import unicodedata

import glob

import datetime
import gmaps

from pathlib import Path

import numpy as np
import pandas as pd

import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

## Prerequisite

In [2]:
WORKDIR = os.path.abspath(os.getcwd())

## Constants

In [3]:
BRAZILIAN_STATES = [
    { "acronym": "AC", "name": "Acre" },
    { "acronym": "AL", "name": "Alagoas" },
    { "acronym": "AP", "name": "Amapá" },
    { "acronym": "AM", "name": "Amazonas" },
    { "acronym": "BA", "name": "Bahia" },
    { "acronym": "CE", "name": "Ceará" },
    { "acronym": "DF", "name": "Distrito Federal" },
    { "acronym": "ES", "name": "Espírito Santo" },
    { "acronym": "GO", "name": "Goiás" },
    { "acronym": "MA", "name": "Maranhão" },
    { "acronym": "MT", "name": "Mato Grosso" },
    { "acronym": "MS", "name": "Mato Grosso do Sul" },
    { "acronym": "MG", "name": "Minas Gerais" },
    { "acronym": "PA", "name": "Pará" },
    { "acronym": "PB", "name": "Paraíba" },
    { "acronym": "PR", "name": "Paraná" },
    { "acronym": "PE", "name": "Pernambuco" },
    { "acronym": "PI", "name": "Piauí" },
    { "acronym": "RJ", "name": "Rio de Janeiro" },
    { "acronym": "RN", "name": "Rio Grande do Norte" },
    { "acronym": "RS", "name": "Rio Grande do Sul" },
    { "acronym": "RO", "name": "Rondônia" },
    { "acronym": "RR", "name": "Roraima" },
    { "acronym": "SC", "name": "Santa Catarina" },
    { "acronym": "SP", "name": "São Paulo" },
    { "acronym": "SE", "name": "Sergipe" },
    { "acronym": "TO", "name": "Tocantins" }
]

## Get the data

### Hotspot data

__List files used in the analysis__

In [4]:
path = ''.join([WORKDIR, "/data/hotspot/**/*"]) 
hotspot_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [5]:
hotspot_df = pd.concat(map(pd.read_csv, hotspot_files))
hotspot_df.head()

,datahora,satelite,pais,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp
0,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,CURURUPU,Amazonia,0.0,0.8,0.0,-1.87136,-44.78587,NaN
1,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,MARACACUME,Amazonia,0.0,0.1,0.1,-1.82566,-45.88670,NaN
2,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,BURITICUPU,Amazonia,0.0,1.1,0.1,-4.57874,-46.38660,NaN
3,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,PAULO RAMOS,Amazonia,0.0,1.4,0.1,-4.59554,-45.66039,NaN
4,2018/01/01 04:06:00,NPP-375D,Brasil,MARANHAO,ARAME,Amazonia,0.0,0.4,0.3,-5.21960,-46.12886,NaN


### Amazon SPI data

__List files used in the analysis__

In [6]:
path = ''.join([WORKDIR, "/data/spi/amazonia/detailed"])
spi_files = glob.glob(os.path.join(path, "*.csv"))

__Load into Pandas DataFrame__

In [7]:
spi_df = pd.concat(map(pd.read_csv, spi_files))
spi_df.head()

,Ano,Código IBGE,Município,Estado,IPS Amazônia,Ranking IPS,Necessidades Humanas Básicas,Fundamentos para o Bem-Estar,Oportunidades,Nutrição e cuidados médicos básicos,...,"Desmatamento recente 2019, 2019, 2020 (% área total do município)",Focos de calor 2020 (nª de focos/1.000 habitantes),Diversidade Partidária 2020 (% vereadores eleitos partidos diferentes),Transporte Público 2020 (nº de ônibus e micro-ônibus/1.000 habitantes),"Acesso à cultura, esporte e lazer 2018 (Categórica 1-10)",Gravidez na infância e adolescência 2019 (% de filhos de mães com até 19 anos),Trabalho Infantil 2019 (nº de famílias com ao menos 1 membro em trabalho infantil/1.000 famílias),Vulnerabilidade familiar 2019 (% de filhos de mães solteiras),Empregos ensino superior 2019 (% de empregos em relação ao total),Mulheres com empregos ensino superior 2019 (% de empregos em relação ao total)
0,2014,1100015.0,Alta Floresta D'Oeste,RO,56.594170,197.0,60.195511,57.369340,52.217659,95.154708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,1100023.0,Ariquemes,RO,55.728511,264.0,59.059533,59.304304,48.821695,93.920755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1100031.0,Cabixi,RO,58.915724,90.0,74.178132,54.787265,47.781775,88.092664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,1100049.0,Cacoal,RO,61.717654,28.0,72.691093,60.613765,51.848105,91.956107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,1100056.0,Cerejeiras,RO,54.593926,361.0,62.606094,62.512401,38.663282,87.810163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Explore the data

### Describe the data I

__Hotspot__

In [8]:
hotspot_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
diasemchuva,16943121.0,13.348092,65.530664,-999.00000,2.00000,5.000000,20.00000,120.00
precipitacao,16943121.0,0.859245,3.598378,0.00000,0.00000,0.000000,0.10000,203.70
riscofogo,16943121.0,-5.579153,79.406999,-999.00000,0.60000,1.000000,1.00000,1.00
latitude,19706174.0,-8.260597,4.323302,-18.03900,-11.07466,-8.680406,-5.30000,5.23
longitude,19706174.0,-53.774619,6.601975,-73.93146,-58.20700,-53.091000,-48.23736,-41.80
frp,5140138.0,19.285664,56.710458,-3.70000,3.30000,7.400000,17.10000,9722.60


__Amazon SPI__

In [9]:
spi_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Ano,2319.0,2.017667e+03,2.868060e+00,2.014000e+03,2.014000e+03,2.018000e+03,2.021000e+03,2.021000e+03
Código IBGE,2316.0,2.288414e+06,1.365403e+06,1.100015e+06,1.502798e+06,1.713254e+06,2.109278e+06,5.108956e+06
IPS Amazônia,2316.0,5.456015e+01,3.811311e+00,4.359790e+01,5.192879e+01,5.430994e+01,5.680819e+01,7.441633e+01
Ranking IPS,2316.0,3.865000e+02,2.229051e+02,1.000000e+00,1.937500e+02,3.865000e+02,5.792500e+02,7.720000e+02
Necessidades Humanas Básicas,2319.0,6.437410e+01,6.825254e+00,4.337849e+01,5.969696e+01,6.432076e+01,6.887852e+01,8.712833e+01
...,...,...,...,...,...,...,...,...
Gravidez na infância e adolescência 2019 (% de filhos de mães com até 19 anos),772.0,2.384973e+01,6.295921e+00,5.172414e+00,1.955196e+01,2.412254e+01,2.815534e+01,4.545455e+01
Trabalho Infantil 2019 (nº de famílias com ao menos 1 membro em trabalho infantil/1.000 famílias),772.0,3.170346e+01,6.065901e+01,0.000000e+00,2.748239e+00,9.631206e+00,3.342557e+01,4.740000e+02
Vulnerabilidade familiar 2019 (% de filhos de mães solteiras),772.0,4.370381e+01,2.095850e+01,5.000000e+00,2.629215e+01,3.899752e+01,5.949844e+01,9.534161e+01
Empregos ensino superior 2019 (% de empregos em relação ao total),772.0,1.828965e+01,1.281655e+01,4.964422e+00,1.129523e+01,1.595505e+01,2.173583e+01,1.479940e+02


### Get information about data

__Hotspot__

In [10]:
hotspot_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19706174 entries, 0 to 318737
Data columns (total 12 columns):
 #   Column        Dtype  
---  ------        -----  
 0   datahora      object 
 1   satelite      object 
 2   pais          object 
 3   estado        object 
 4   municipio     object 
 5   bioma         object 
 6   diasemchuva   float64
 7   precipitacao  float64
 8   riscofogo     float64
 9   latitude      float64
 10  longitude     float64
 11  frp           float64
dtypes: float64(6), object(6)
memory usage: 1.9+ GB


__Amazon SPI__

In [11]:
spi_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2319 entries, 0 to 772
Columns: 158 entries, Ano to Mulheres com empregos ensino superior 2019 (% de empregos em relação ao total)
dtypes: float64(155), int64(1), object(2)
memory usage: 2.8+ MB


### Percentage of missing data

In [12]:
def get_pct_missing_data(dataset):
    """
    Get Percentage of missing data
    
    Attributes
    ----------
    dataset : Pandas DataFrame
    """
    total = dataset.isnull().sum().sort_values(ascending=False)
    percent = dataset.isnull().sum() / dataset.isnull().count() * 100
    percent = (round(percent, 4)).sort_values(ascending=False)

    missing_data = pd.concat([total, percent], keys=["Total", '%'], axis=1)

    return missing_data

__Hotspot__

In [13]:
get_pct_missing_data(hotspot_df)

,Total,%
frp,14566036,73.9161
diasemchuva,2763053,14.0213
precipitacao,2763053,14.0213
riscofogo,2763053,14.0213
datahora,0,0.0000
satelite,0,0.0000
pais,0,0.0000
estado,0,0.0000
municipio,0,0.0000
bioma,0,0.0000


__Amazon SPI__

In [14]:
get_pct_missing_data(spi_df)

,Total,%
Moradias com iluminação adequada 2018 (% de domicilios),1547,66.7098
Vulnerabilidade familiar 2017 (% de filhos de mães solteiras),1547,66.7098
Desmatamento acumulado 2017 (% área total do município),1547,66.7098
"Desmatamento recente 2017, 2017, 2018 (% área total do município)",1547,66.7098
Focos de calor 2018 (nª de focos/1.000 habitantes),1547,66.7098
...,...,...
Segurança pessoal,0,0.0000
Acesso ao conhecimento básico,0,0.0000
Acesso à informação e comunicação,0,0.0000
Saúde e bem-estar,0,0.0000


## Prepare the data

### Standardize Country State Names

The feature __estado__ in the two datasets are diffenrents. In _hotspot_df_ the name of state is long and spi_df is short.

__Amazon SPI__

In [15]:
def get_full_name_state(acronym):
    """
    Retrieves the long name of state related to acronym.
    """
    lnames = [x.get('name') for x in BRAZILIAN_STATES if x.get('acronym') == acronym]
    return lnames[0] if lnames else np.nan


def purge_spec_chars(word):
    """
    Remove all special characters in the word.
    """
    wv = word if word is not np.nan else ""
    nfkd_form = unicodedata.normalize("NFKD", wv) 
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [16]:
# Get full name of state
spi_df["Estado"] = spi_df["Estado"].apply(get_full_name_state)

# Remove special characteres
spi_df["Estado"] = spi_df["Estado"].apply(purge_spec_chars)

# Capitalize the name
spi_df["Estado"] = spi_df["Estado"].apply(lambda x: x.upper())

### Standardize Cities Names

The feature __municipio__ in the two datasets are diffenrents.

In [17]:
# Remove special characteres
spi_df["Município"] = spi_df["Município"].apply(purge_spec_chars)

# Capitalize the name
spi_df["Município"] = spi_df["Município"].apply(lambda x: x.upper())

#### Some city names are written incorrectly. In this section I will fix it with the correct form

__hotspot__

In [18]:
hotspot_df.loc[(hotspot_df["municipio"] == "ELDORADO DO CARAJAS"), "municipio"] = "ELDORADO DOS CARAJAS"

__Amazon SPI__

In [19]:
# spi_df.loc[spi_df["Município"] == "BARREIRINHA", "Município"] = "BARREIRINHAS"

# Acre
spi_df.loc[spi_df["Município"] == "RIO BRANCO (ACRE)", "Município"] = "RIO BRANCO"

# Maranhão
spi_df.loc[spi_df["Município"] == "ARAGUANA (MARANHAO)", "Município"] = "ARAGUANA"
spi_df.loc[spi_df["Município"] == "PRESIDENTE MEDICI (MARANHAO)", "Município"] = "PRESIDENTE MEDICI"

# Mato Grosso
spi_df.loc[spi_df["Município"] == "POXOREO", "Município"] = "POXOREU"
spi_df.loc[spi_df["Município"] == "RIO BRANCO (MATO GROSSO)", "Município"] = "RIO BRANCO"

# Rondonia
spi_df.loc[spi_df["Município"] == "PRESIDENTE MEDICI (RONDONIA)", "Município"] = "PRESIDENTE MEDICI"

# Pará
spi_df.loc[spi_df["Município"] == "BOM JESUS DO TOCANTINS (PARA)", "Município"] = "BOM JESUS DO TOCANTINS"
spi_df.loc[spi_df["Município"] == "SANTA ISABEL DO PARA", "Município"] = "SANTA IZABEL DO PARA"

# Tocantins
spi_df.loc[spi_df["Município"] == "ARAGUANA (TOCANTINS)", "Município"] = "ARAGUANA"
spi_df.loc[spi_df["Município"] == "BOM JESUS DO TOCANTINS (TOCANTINS)", "Município"] = "BOM JESUS DO TOCANTINS"
spi_df.loc[spi_df["Município"] == "COUTO DE MAGALHAES", "Município"] = "COUTO MAGALHAES"
spi_df.loc[spi_df["Município"] == "SAO VALERIO DA NATIVIDADE", "Município"] = "SAO VALERIO"

#### Shows the difference between the number of cities in the two datasets

In [20]:
hotspot_regions = hotspot_df.groupby(['municipio', 'estado'])
spi_region = spi_df.groupby(['Município', 'Estado'])

n_hr = len(hotspot_regions)
n_sr = len(spi_region)

dic_cities = set(hotspot_regions.groups.keys()) ^ set(spi_region.groups.keys())

print("hotspot_df has {} cities and spi_df has {}. The Difference is {}.".format(n_hr, n_sr, (n_hr - n_sr)))
print("---------------------------------------------------------------------------------------------------")
print("Cities not found:\n{}".format(dic_cities))

hotspot_df has 808 cities and spi_df has 773. The Difference is 35.
---------------------------------------------------------------------------------------------------
Cities not found:
{('PRIMEIRA CRUZ', 'MARANHAO'), ('CHAPADINHA', 'MARANHAO'), ('MATA ROMA', 'MARANHAO'), ('TUTOIA', 'MARANHAO'), ('', ''), ('ALDEIAS ALTAS', 'MARANHAO'), ('PAULINO NEVES', 'MARANHAO'), ('SANTA QUITERIA DO MARANHAO', 'MARANHAO'), ('SANTO AMARO DO MARANHAO', 'MARANHAO'), ('DUQUE BACELAR', 'MARANHAO'), ('CAXIAS', 'MARANHAO'), ('ARAIOSES', 'MARANHAO'), ('URBANO SANTOS', 'MARANHAO'), ('TIMON', 'MARANHAO'), ('MATOES', 'MARANHAO'), ('LAGOA DO MATO', 'MARANHAO'), ('BREJO', 'MARANHAO'), ('AGUA DOCE DO MARANHAO', 'MARANHAO'), ('AFONSO CUNHA', 'MARANHAO'), ('MILAGRES DO MARANHAO', 'MARANHAO'), ('HUMBERTO DE CAMPOS', 'MARANHAO'), ('ANAPURUS', 'MARANHAO'), ('PASSAGEM FRANCA', 'MARANHAO'), ('SAO JOAO DOS PATOS', 'MARANHAO'), ('BARAO DE GRAJAU', 'MARANHAO'), ('COELHO NETO', 'MARANHAO'), ('SAO BERNARDO', 'MARANHAO'), ('S

In [ ]:
dic_cities_df = pd.DataFrame(dic_cities, columns=["municipio", "estado"])
dic_cities_df = dic_cities_df.dropna()

for index, row in dic_cities_df.iterrows():
    hotspot_df = hotspot_df.drop(hotspot_df[(hotspot_df.municipio == row.municipio) & (hotspot_df.estado == row.estado)].index)

### Parse datehour of string to date

__Hotspot__

In [ ]:
hotspot_df['datahora'] = pd.to_datetime(hotspot_df['datahora'], format='%Y/%m/%d %H:%M:%S')

### Add features

__Hotspot__

In [ ]:
hotspot_df["ano"] = pd.DatetimeIndex(hotspot_df['datahora']).year
# hotspot_df["mes"] = pd.DatetimeIndex(hotspot_df['datahora']).month
# hotspot_df["dia"] = pd.DatetimeIndex(hotspot_df['datahora']).day

### Fix or remove outliers (optional)

__Hotspot__

In [ ]:
hotspot_df.loc[hotspot_df["riscofogo"] < 0, "riscofogo"] = 0
hotspot_df.loc[hotspot_df["diasemchuva"] < 0, "diasemchuva"] = 0
hotspot_df.loc[hotspot_df["frp"] < 0, "frp"] = 0

__Amazon SPI__

In [ ]:
spi_df.dropna(subset=["Município", "Estado", "IPS Amazônia"], inplace=True)

### Fill in missing values (e.g., with zero, mean, median...) or drop their rows (or columns)

__Hotspot__

In [ ]:
hotspot_df["frp"].fillna(value=0, inplace=True)
hotspot_df["riscofogo"].fillna(value=0, inplace=True)
hotspot_df["diasemchuva"].fillna(value=0, inplace=True)
hotspot_df["precipitacao"].fillna(value=0, inplace=True)

### Drop columns not used

__Hotspot__

In [ ]:
hotspot_df = hotspot_df.drop(columns=["datahora", "satelite", "pais", "bioma", "latitude", "longitude"])
hotspot_df.head()

__Amazon SPI__

_Dimensions and their components:_

1. Necessidades Humanas Básicas
    * Nutrição e cuidados médicos básicos
    * Água e saneamento
    * Moradia
    * Segurança pessoal
2. Fundamentos para o Bem-Estar
    * Acesso ao conhecimento básico
    * Acesso à informação e comunicação
    * Saúde e bem-estar",
    * Qualidade do meio ambiente
3. Oportunidades
    * Direitos individuais
    * Liberdade individual e de escolha
    * Tolerância e inclusão
    * Acesso à educação superior

In [ ]:
spi_df = spi_df[[
    "Ano", "Município","Estado","IPS Amazônia","Ranking IPS",
    "Necessidades Humanas Básicas", "Fundamentos para o Bem-Estar","Oportunidades", # 3 Dimensions
    "Nutrição e cuidados médicos básicos","Água e saneamento","Moradia","Segurança pessoal", # Necessidades Humanas Básicas components
    "Acesso ao conhecimento básico", "Acesso à informação e comunicação","Saúde e bem-estar","Qualidade do meio ambiente", # Fundamentos para o Bem-Estar components
    "Direitos individuais","Liberdade individual e de escolha","Tolerância e inclusão","Acesso à educação superior" # Oportunidades components
]]
spi_df.head()

In [ ]:
spi_df.sort_values(["Estado", "Município"])

### Describe the data II

__Hotspot__

In [ ]:
hotspot_df.describe().transpose()

In [ ]:
get_pct_missing_data(hotspot_df)

__Amazon SPI__

In [ ]:
spi_df.describe().transpose()

In [ ]:
get_pct_missing_data(spi_df)

### Data Transform

#### Getting anual mean of each cities by years

__hotspot__

In [ ]:
hotspot_df.loc[hotspot_df.ano < 2014, "ano"] = 2014
hotspot_df.loc[(hotspot_df.ano > 2014) & (hotspot_df.ano < 2018), "ano"] = 2018
hotspot_df.loc[(hotspot_df.ano > 2018) & (hotspot_df.ano < 2021), "ano"] = 2021

In [ ]:
hotspot_df = hotspot_df.groupby(["ano", "estado", "municipio"]).mean()
hotspot_df = hotspot_df.reset_index()
hotspot_df

In [ ]:
output = pd.merge(hotspot_df, spi_df, left_on=["ano","estado", "municipio"], right_on=["Ano", "Estado", "Município"])

In [ ]:
output = result.drop(["Ano", "Município", "Estado"], axis=1)

In [ ]:
output

### Save data in a csv file

In [ ]:
path = Path(f"{WORKDIR}/data/output/hotspot_spi.csv")
         
path.parent.mkdir(parents=True, exist_ok=True)
output.to_csv(path, index=False)